In [1]:
from scipy import stats
srcc = stats.spearmanr( [0.6480761, 0.62233955, 0.6661059, 0.631339, 0.6192408, 0.641552, 0.6423639, 0.7225227, 0.65772206, 0.63077545, 0.54655486  , 0.6763791, 0.62823856, 0.63159883, 0.62590545, 0.65006614, 0.6068853, 0.63621056, 0.5723472, 0.6442189, 0.62065214, 0.67560524, 0.6549503, 0.61165047], [0.062, 0.032, 0.029, 0.028, 0.027, 0.104, 0.028, 0.006, 0.033, 0.088, 0.052, 0.014, 0.029, 0.032, 0.022, 0.0, 0.012, 0.04, 0.051, 0.003, 0.032, 0.031, 0.019, 0.042])
srcc[0]

-0.3369615411989142

In [2]:
import torch
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())


NVIDIA GeForce GTX 1050
True


In [3]:
%cd ..


d:\quality_assessment\knowledge_distillation\cvr-iqa\CVRKD-IQA


In [4]:
from test_DistillationIQA_single import cvr_on_single_image



normal mode...
>>>>>  ./model_zoo/NAR_student_cross_dataset.pth


In [5]:

cvr_on_single_image("tst_imgs/i01_19_4.bmp" , "tst_imgs/i01_23_1.bmp")

2.545132875442505

# Correlation


# dst:

In [6]:
import glob

TID_ADDR = "D:\quality_assessment\knowledge_distillation\cvr-iqa\CVRKD-IQA\dataset\TID2013"
dsts = ["01","02","03","04","05","06","07","08","09" , "10", "11", "12", "13", "14", "15", "16", "17", "18", "19","20","21","22","23","24"]


def get_images_dst(dst_type):
    imgs = glob.glob(TID_ADDR+'/distorted_images/*_'+ dst_type +'_'+ '*' +'.*')
    return imgs

def get_mos_dst(dst_type ):
    mos = []
    f = open(TID_ADDR+"/mos_with_names.txt", "r")
    lines = f.readlines()
    for l in lines:
        if l.split(' ')[1].split('_')[1] == dst_type:
            mos.append(float(l.split(' ')[0][:5]))

    return mos


refs = glob.glob(TID_ADDR+'/reference_images/*')

refs_for_dst = []

for elem in refs:
    for i in range(5):
        refs_for_dst.append(elem)

In [7]:
import random , glob

def get_random_HR_images():
    all_images = glob.glob('D:\quality_assessment\knowledge_distillation\cvr-iqa\CVRKD-IQA\dataset\DIV2K_ref/val_HR/*')
    return random.sample(all_images, 1)[0]


get_random_HR_images()

'D:\\quality_assessment\\knowledge_distillation\\cvr-iqa\\CVRKD-IQA\\dataset\\DIV2K_ref/val_HR\\0861.png'

In [8]:
import xlsxwriter
 
workbook = xlsxwriter.Workbook('SRCC_on_DST-Content-variant-ref(normal-mode).xlsx')
worksheet = workbook.add_worksheet()


row = 3


for c in dsts:
    imgs = get_images_dst(c)
    # refs = refs_for_dst
    cosines = []
    moses = get_mos_dst(c)
    for i in range(len(imgs)):
        score = cvr_on_single_image(imgs[i] , get_random_HR_images())
        
        cosines.append(score)

    worksheet.write(row, 3,  c)
    worksheet.write(row, 4,  stats.spearmanr( cosines , moses)[0])

    
    print(row)
    row +=1

workbook.close()  

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


all imgs


In [9]:
imgs = []
refs = []
moses = []
for c in dsts:
    imgs += get_images_dst(c)
    refs += get_random_HR_images()
    moses += get_mos_dst(c)


cosines = []
for i in range(len(imgs)):
    rr = cvr_on_single_image(imgs[i] , get_random_HR_images())
    
    cosines.append( rr )


print(stats.spearmanr( cosines , moses)[0])

0.691959491885982


content

In [9]:
contents = ["01","02","03","04","05","06","07","08","09" , "10", "11", "12", "13", "14", "15", "16", "17", "18", "19","20","21","22","23","24","25"]


def get_images_content(im_name):
    imgs = glob.glob(TID_ADDR+'/distorted_images/*'+im_name+'_'+ '*' +'_'+ '*' +'.*')
    return imgs

def get_mos_content(im_name ):
    mos = []
    f = open(TID_ADDR+"/mos_with_names.txt", "r")
    lines = f.readlines()
    for l in lines:
        if l.split(' ')[1][1:3] == im_name:
            mos.append(float(l.split(' ')[0][:5]))

    return mos


def get_refs_content(content):
        return glob.glob(TID_ADDR+'/reference_images/*'+content+'*') * 120





import xlsxwriter
 
workbook = xlsxwriter.Workbook('SRCC_cvr-contents-originalRefs(stacking).xlsx')
worksheet = workbook.add_worksheet()



row = 3


for c in contents:
    imgs = get_images_content(c)
    refs = get_refs_content(c)
    cosines = []
    moses = get_mos_content(c)
    for i in range(len(imgs)):
        dd = cvr_on_single_image(imgs[i],refs[i])
        
        cosines.append( dd)

    worksheet.write(row, 3,  c)
    worksheet.write(row, 4,  stats.spearmanr( cosines , moses)[0])

    
    print(row)
    row +=1

workbook.close() 

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


all imgs

In [10]:
imgs = []
refs = []
moses = []
for c in contents:
    imgs += get_images_content(c)
    refs += get_refs_content(c)
    moses += get_mos_content(c)


cosines = []
for i in range(len(imgs)):
    dd = cvr_on_single_image(imgs[i] , refs[i])
    
    cosines.append(dd)

stats.spearmanr( cosines , moses)[0]

0.7528909932773492